In [27]:
!pip install sidrapy
import pandas as pd
import zipfile
import io
import sidrapy
import geopandas as gpd

In [28]:
# Coletar o IPCA (índice mensal, base: Dezembro de 1993 = 100)
try:
    ipca_raw = sidrapy.get_table(
        table_code="1737",
        territorial_level="1",
        ibge_territorial_code="all",
        variable="2266",  # IPCA com número-índice
        period="all"
    )

    print(ipca_raw)

    # Limpeza inicial
    df_ipca = pd.DataFrame(ipca_raw)

    df_ipca.columns = df_ipca.iloc[0]
    df_ipca = df_ipca[1:].reset_index(drop=True)
    df_ipca_mensal = df_ipca[['Mês (Código)', 'Valor']].copy()
    df_ipca_mensal.columns = ['mes_codigo', 'ipca']

    # Converter para data e valor numérico
    df_ipca_mensal['data'] = pd.to_datetime(df_ipca_mensal['mes_codigo'], format='%Y%m')
    df_ipca_mensal['ipca'] = pd.to_numeric(df_ipca_mensal['ipca'], errors='coerce')
    df_ipca_mensal['ano'] = df_ipca_mensal['data'].dt.year

    # Calcular a média anual do IPCA
    df_ipca_anual = df_ipca_mensal.groupby('ano')['ipca'].mean().reset_index()
    df_ipca_anual.rename(columns={'ipca': 'ipca_medio_anual'}, inplace=True)

    # Obter o valor do IPCA para o mês base (Dezembro/2021)
    ipca_base_dez_2021 = df_ipca_mensal[df_ipca_mensal['data'] == '2021-12-01']['ipca'].iloc[0]

    print("IPCA coletado e processado com sucesso.")
    print(f"Valor do IPCA base (Dez/2021): {ipca_base_dez_2021}")

except Exception as e:
    print(f"Erro ao coletar o IPCA: {e}")

                             NC                 NN  \
0    Nível Territorial (Código)  Nível Territorial   
1                             1             Brasil   
2                             1             Brasil   
3                             1             Brasil   
4                             1             Brasil   
..                          ...                ...   
543                           1             Brasil   
544                           1             Brasil   
545                           1             Brasil   
546                           1             Brasil   
547                           1             Brasil   

                             MC                 MN                   V  \
0    Unidade de Medida (Código)  Unidade de Medida               Valor   
1                            30      Número-índice     0.0000000076183   
2                            30      Número-índice     0.0000000081223   
3                            30      Número-índice     

In [29]:
zip_files = [
    '/content/drive/MyDrive/02 - Economia (UFPE)/2025.1/Tópicos de Macro (Economia do clima)/analise-saneamento-chuva-extrema-pe/build/input/pib-pop-ibge/base_de_dados_2002_2009_xls.zip',
    '/content/drive/MyDrive/02 - Economia (UFPE)/2025.1/Tópicos de Macro (Economia do clima)/analise-saneamento-chuva-extrema-pe/build/input/pib-pop-ibge/base_de_dados_2010_2021_xlsx.zip'
]

# Lista para armazenar os DataFrames de cada arquivo lido
lista_de_dfs = []

print("Iniciando a extração e leitura dos arquivos...")

# Passo 3: Loop para ler os arquivos de dentro de cada ZIP
for file in zip_files:
    print(f"Processando o arquivo: {file}")
    with zipfile.ZipFile(file, 'r') as archive:
        # Loop para ler cada arquivo dentro do ZIP
        for sub_file in archive.namelist():
            # Garantir que estamos lendo apenas arquivos Excel
            if sub_file.endswith(('.xls', '.xlsx')):
                print(f"  Lendo sub-arquivo: {sub_file}")

                # Ler o arquivo Excel diretamente do ZIP para a memória
                with archive.open(sub_file) as f:
                    # Ler o conteúdo do arquivo em bytes
                    excel_data = io.BytesIO(f.read())

                    # Usar o engine correto dependendo da extensão do arquivo
                    engine = 'xlrd' if sub_file.endswith('.xls') else 'openpyxl'

                    # Ler o arquivo Excel. A opção 'header=1' é comum em arquivos do IBGE
                    # que têm um título na primeira linha. Ajuste se necessário.
                    df_temp = pd.read_excel(excel_data, engine=engine, header=0)
                    lista_de_dfs.append(df_temp)

print("Leitura concluída. Concatenando os dados...")

# ---
# Passo 4: Concatenar todos os DataFrames em um só
df_completo = pd.concat(lista_de_dfs, ignore_index=True)

print("Dados concatenados. Filtrando para Pernambuco...")

# Passo 5: Filtrar para manter apenas os municípios de Pernambuco (PE)
# É importante verificar o nome exato da coluna da UF no arquivo
# Com base nos arquivos do IBGE, o nome costuma ser 'Sigla da Unidade da Federação'
df_pe = df_completo[df_completo['Sigla da Unidade da Federação'] == 'PE'].copy()

print("Filtro aplicado. Selecionando e renomeando colunas...")

# Passo 6: Selecionar e renomear as colunas de interesse

# Nomes exatos das colunas como estão no arquivo
coluna_pib_total = 'Produto Interno Bruto, \na preços correntes\n(R$ 1.000)'
coluna_pib_per_capita = 'Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)'

# Dicionário para renomear
colunas_interesse = {
    'Ano': 'ano',
    'Nome do Município': 'nome_municipio',
    'Código do Município': 'cod_municipio',
    coluna_pib_total: 'pib_mil_brl',
    coluna_pib_per_capita: 'pib_per_capita'
}

df_pe_final = df_pe[colunas_interesse.keys()].rename(columns=colunas_interesse)

print("Colunas selecionadas. Calculando a população residente estimada...")

# ---
# Passo 7: Calcular a população residente
# Primeiro, garantir que as colunas são numéricas e tratar possíveis erros
df_pe_final['pib_mil_brl'] = pd.to_numeric(df_pe_final['pib_mil_brl'], errors='coerce')
df_pe_final['pib_per_capita'] = pd.to_numeric(df_pe_final['pib_per_capita'], errors='coerce')

# Remover linhas onde a conversão para número falhou ou onde os valores são zero/nulos
df_pe_final.dropna(subset=['pib_mil_brl', 'pib_per_capita'], inplace=True)
df_pe_final = df_pe_final[df_pe_final['pib_per_capita'] > 0]

# Ajustar a unidade do PIB total (de R$ 1.000 para R$ 1)
pib_total_brl = df_pe_final['pib_mil_brl'] * 1000

# Calcular a população e arredondar para o inteiro mais próximo
df_pe_final['populacao_estimada'] = (pib_total_brl / df_pe_final['pib_per_capita']).round(0)

print("\n--- Processamento Concluído! ---")
print("Amostra da base de dados final para Pernambuco:")
print(df_pe_final.head())

print("\nInformações sobre a base de dados final:")
df_pe_final.info()

# ---
print("\n--- Iniciando de união do PIB e IPCA ---")
df_pe_final['ano'] = pd.to_numeric(df_pe_final['ano'])
df_deflacao = pd.merge(df_pe_final, df_ipca_anual, on='ano', how='left')

# ---
# DEFLACIONAR O PIB E CALCULAR PIB PER CAPITA REAL
# ---
print("\n--- Iniciando Etapa 4: Deflacionando o PIB para preços de Dez/2021 ---")
# Fórmula do Deflacionamento: ValorReal = ValorCorrente * (IndiceBase / IndiceCorrente)
df_deflacao['pib_mil_brl_deflacionado'] = df_deflacao['pib_mil_brl'] * (ipca_base_dez_2021 / df_deflacao['ipca_medio_anual'])

# Calcular o novo PIB per capita com o valor deflacionado
df_deflacao['pib_per_capita_deflacionado'] = ((df_deflacao['pib_mil_brl_deflacionado'] * 1000) / df_deflacao['populacao_estimada']).round(2)
print("Cálculos de deflacionamento concluídos.")

# Selecionar as colunas finais e mais importantes
colunas_finais = [
    'cod_municipio',
    'ano',
    'populacao_estimada',
    'pib_per_capita_deflacionado'
]
df_final_socioeconomico = df_deflacao[colunas_finais]

print("Amostra do DataFrame final com PIB deflacionado:")
print(df_final_socioeconomico.head())
df_final_socioeconomico.info()

Iniciando a extração e leitura dos arquivos...
Processando o arquivo: /content/drive/MyDrive/02 - Economia (UFPE)/2025.1/Tópicos de Macro (Economia do clima)/analise-saneamento-chuva-extrema-pe/build/input/pib-pop-ibge/base_de_dados_2002_2009_xls.zip
  Lendo sub-arquivo: PIB dos Municípios - base de dados 2002-2009.xls
Processando o arquivo: /content/drive/MyDrive/02 - Economia (UFPE)/2025.1/Tópicos de Macro (Economia do clima)/analise-saneamento-chuva-extrema-pe/build/input/pib-pop-ibge/base_de_dados_2010_2021_xlsx.zip
  Lendo sub-arquivo: PIB dos Municípios - base de dados 2010-2021.xlsx
Leitura concluída. Concatenando os dados...
Dados concatenados. Filtrando para Pernambuco...
Filtro aplicado. Selecionando e renomeando colunas...
Colunas selecionadas. Calculando a população residente estimada...

--- Processamento Concluído! ---
Amostra da base de dados final para Pernambuco:
       ano         nome_municipio  cod_municipio  pib_mil_brl  pib_per_capita  \
1462  2002           Abr

In [30]:
# ETAPA: CÁLCULO DA DENSIDADE POPULACIONAL
# ---
print("\n--- Iniciando o Cálculo da Densidade Populacional ---")

# 1. Defina o caminho para o seu arquivo shapefile (dentro do ZIP)
caminho_shapefile = "/content/drive/MyDrive/02 - Economia (UFPE)/2025.1/Tópicos de Macro (Economia do clima)/analise-saneamento-chuva-extrema-pe/build/input/PE_Municipios_2024.zip"

print(f"Lendo o shapefile de: {caminho_shapefile}")
try:
    # 2. Carregar o shapefile usando o geopandas
    # COMENTÁRIO: Esta linha está perfeita. O geopandas vai abrir o ZIP e ler o shapefile.
    gdf_pe = gpd.read_file(caminho_shapefile)

    # 3. Calcular a área dos municípios
    # COMENTÁRIO: A lógica de reprojetar o CRS para calcular a área é a abordagem correta e mais precisa.
    # O código EPSG 31984 (SIRGAS 2000 / UTM Zone 24S) é adequado para a maior parte de Pernambuco.
    gdf_pe_utm = gdf_pe.to_crs(epsg=4674)

    print("\nCálculo de área concluído com sucesso. Amostra dos dados:")
    print(gdf_pe[['NM_MUN', 'AREA_KM2']].head())

except Exception as e:
    print(f"Ocorreu um erro: {e}")


--- Iniciando o Cálculo da Densidade Populacional ---
Lendo o shapefile de: /content/drive/MyDrive/02 - Economia (UFPE)/2025.1/Tópicos de Macro (Economia do clima)/analise-saneamento-chuva-extrema-pe/build/input/PE_Municipios_2024.zip

Cálculo de área concluído com sucesso. Amostra dos dados:
            NM_MUN  AREA_KM2
0      Rio Formoso   227.458
1     Bom Conselho   790.774
2          Panelas   380.428
3  Santa Terezinha   200.327
4          Quipapá   230.617


In [31]:
# 4. Preparar o DataFrame de área para o merge
# Selecionamos apenas o código do município e a área
# O nome da coluna do código no shapefile do IBGE é geralmente 'CD_MUN'
df_area = gdf_pe[['CD_MUN', 'AREA_KM2']].copy()
df_area.rename(columns={'CD_MUN': 'cod_municipio'}, inplace=True)

# Garantir que o tipo de dado da chave é o mesmo
df_area['cod_municipio'] = df_area['cod_municipio'].astype('int64')

# 5. Unir (merge) a informação de área ao seu DataFrame socioeconômico
df_final_com_densidade = pd.merge(
        df_final_socioeconomico,
        df_area,
        on='cod_municipio',
        how='left'
    )

# 6. Calcular a densidade populacional
df_final_com_densidade['densidade_populacional'] = (
df_final_com_densidade['populacao_estimada'] / df_final_com_densidade['AREA_KM2']
    ).round(2)
df_final_com_densidade = df_final_com_densidade.drop(columns=['AREA_KM2'])
print("Cálculo da densidade populacional concluído.")


df_final_com_densidade.to_parquet('/content/drive/MyDrive/02 - Economia (UFPE)/2025.1/Tópicos de Macro (Economia do clima)/analise-saneamento-chuva-extrema-pe/build/output/pib_dens_hab_pe_2002_2021.parquet', index=False)

Cálculo da densidade populacional concluído.
